# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse Shipments        Customer  \
0  Dec 30 2022 11:32AM  254057        19   8968258  Innogenix, LLC   
1  Dec 30 2022 11:32AM  254057        19   8968259  Innogenix, LLC   
2  Dec 30 2022 11:32AM  254057        19   8968260  Innogenix, LLC   
3  Dec 30 2022 11:32AM  254057        19   8968261  Innogenix, LLC   
4  Dec 30 2022 11:32AM  254057        19   8968262  Innogenix, LLC   
5  Dec 30 2022 11:32AM  254057        19   8968263  Innogenix, LLC   
6  Dec 30 2022 11:32AM  254057        19   8968264  Innogenix, LLC   
7  Dec 30 2022 11:32AM  254057        19   8968265  Innogenix, LLC   
8  Dec 30 2022 11:32AM  254057        19   8968266  Innogenix, LLC   
9  Dec 30 2022 11:32AM  254057        19   8968267  Innogenix, LLC   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
15  254051       Released          1
16  254052       Released          1
17  254053       Released          1
18  254055       Released          1
19  254057       Released         23

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
254051                NaN        NaN       1.0
254052                NaN        NaN       1.0
254053                NaN        NaN       1.0
254055                NaN        NaN       1.0
254057                NaN        NaN      23.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
254008                0.0        0.0       3.0
254016                0.0        0.0       3.0
254027                0.0        0.0       1.0
254031                0.0        0.0       1.0
254037                9.0       18.0      14.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
254008                  0          0         3
254016                  0          0         3
254027                  0          0         1
254031                  0          0         1
254037                  9         18        14

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               254008          0          0         3
1               254016          0          0         3
2               254027          0          0         1
3               254031          0          0         1
4               254037          9         18        14

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing  Released
0               254008                             3
1               254016                             3
2               254027                             1
3               254031                             1
4               254037         9        18        14

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                       Customer
0   Dec 30 2022 11:32AM  254057        19                 Innogenix, LLC
23  Dec 30 2022 11:18AM  254055        10                    Emerginnova
24  Dec 30 2022 10:53AM  254050        10               Eywa Pharma Inc.
27  Dec 30 2022 10:46AM  254053        19          ACG North America LLC
28  Dec 30 2022 10:35AM  254052        12                      Hush Hush
29  Dec 30 2022 10:29AM  254051        10  Nextsource Biotechnology, LLC
30  Dec 30 2022 10:06AM  254048        10                        Bio-PRF
40  Dec 30 2022 10:05AM  254049        10                        Bio-PRF
48  Dec 30 2022  9:13AM  254046        12                  Livs Products
49  Dec 30 2022  9:08AM  254045        10                      SugarBear

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                       Customer  \
0  Dec 30 2022 11:32AM  254057        19                 Innogenix, LLC   
1  Dec 30 2022 11:18AM  254055        10                    Emerginnova   
2  Dec 30 2022 10:53AM  254050        10               Eywa Pharma Inc.   
3  Dec 30 2022 10:46AM  254053        19          ACG North America LLC   
4  Dec 30 2022 10:35AM  254052        12                      Hush Hush   
5  Dec 30 2022 10:29AM  254051        10  Nextsource Biotechnology, LLC   
6  Dec 30 2022 10:06AM  254048        10                        Bio-PRF   
7  Dec 30 2022 10:05AM  254049        10                        Bio-PRF   
8  Dec 30 2022  9:13AM  254046        12                  Livs Products   
9  Dec 30 2022  9:08AM  254045        10                      SugarBear   

  Completed Executing  Released  
0                            23  
1                             1  
2                             3  
3                             1  
4                             1  
5                             1  
6                            10  
7                             8  
8                             1  
9                             1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                       Customer  \
0  Dec 30 2022 11:32AM  254057        19                 Innogenix, LLC   
1  Dec 30 2022 11:18AM  254055        10                    Emerginnova   
2  Dec 30 2022 10:53AM  254050        10               Eywa Pharma Inc.   
3  Dec 30 2022 10:46AM  254053        19          ACG North America LLC   
4  Dec 30 2022 10:35AM  254052        12                      Hush Hush   
5  Dec 30 2022 10:29AM  254051        10  Nextsource Biotechnology, LLC   
6  Dec 30 2022 10:06AM  254048        10                        Bio-PRF   
7  Dec 30 2022 10:05AM  254049        10                        Bio-PRF   
8  Dec 30 2022  9:13AM  254046        12                  Livs Products   
9  Dec 30 2022  9:08AM  254045        10                      SugarBear   

   Released Executing Completed  
0        23                      
1         1                      
2         3                      
3         1                      
4         1                      
5         1                      
6        10                      
7         8                      
8         1                      
9         1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Dec 30 2022 11:32AM  254057        19         Innogenix, LLC        23   
1  Dec 30 2022 11:18AM  254055        10            Emerginnova         1   
2  Dec 30 2022 10:53AM  254050        10       Eywa Pharma Inc.         3   
3  Dec 30 2022 10:46AM  254053        19  ACG North America LLC         1   
4  Dec 30 2022 10:35AM  254052        12              Hush Hush         1   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Dec 30 2022 11:32AM  254057        19         Innogenix, LLC        23   
1  Dec 30 2022 11:18AM  254055        10            Emerginnova         1   
2  Dec 30 2022 10:53AM  254050        10       Eywa Pharma Inc.         3   
3  Dec 30 2022 10:46AM  254053        19  ACG North America LLC         1   
4  Dec 30 2022 10:35AM  254052        12              Hush Hush         1   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Dec 30 2022 11:32AM  254057        19         Innogenix, LLC        23   
1  Dec 30 2022 11:18AM  254055        10            Emerginnova         1   
2  Dec 30 2022 10:53AM  254050        10       Eywa Pharma Inc.         3   
3  Dec 30 2022 10:46AM  254053        19  ACG North America LLC         1   
4  Dec 30 2022 10:35AM  254052        12              Hush Hush         1   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3048460        34        0.0        0.0
12          508098         2        0.0        0.0
16          254042         1        0.0        0.0
19          762147        38       18.0        9.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3048460        34        0.0        0.0
1        12   508098         2        0.0        0.0
2        16   254042         1        0.0        0.0
3        19   762147        38       18.0        9.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10        34        0.0        0.0
1        12         2        0.0        0.0
2        16         1        0.0        0.0
3        19        38       18.0        9.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse     Status  Value
0        10   Released   34.0
1        12   Released    2.0
2        16   Released    1.0
3        19   Released   38.0
4        10  Executing    0.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12   16    19
Status                         
Completed   0.0  0.0  0.0   9.0
Executing   0.0  0.0  0.0  18.0
Released   34.0  2.0  1.0  38.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12   16    19
0          Completed   0.0  0.0  0.0   9.0
1          Executing   0.0  0.0  0.0  18.0
2           Released  34.0  2.0  1.0  38.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12   16    19
0  Completed   0.0  0.0  0.0   9.0
1  Executing   0.0  0.0  0.0  18.0
2   Released  34.0  2.0  1.0  38.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()